# Performance Comparison: Text Classification Transfer Learning with Hugging Face* and the Intel® Transfer Learning Tool

This notebook uses the Hugging Face Trainer to do transfer learning with a text classification model with PyTorch*. The model is trained, evaluated, and exported. The same sequence is also done using the Intel Transfer Learning Tool. The Intel Transfer Learning Tool has a flag to control whether the Hugging Face Trainer is used under the hood, or if just PyTorch libraries are used. Training and evaluation are run both ways, giving us three combinations to compare:
* Using the Hugging Face Trainer
* Using the Intel Transfer Learning Tool with the Hugging Face Trainer
* Using the Intel Transfer Learning Tool with torch

After all of the models have been trained and evaluated, charts are displayed to visually compare the training and evaluation metrics.

In [ ]:
import os
import psutil

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import datasets
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
import torch
import transformers
from transformers import DataCollatorWithPadding, Trainer
from torch.utils.data import DataLoader as loader

from tlt.datasets import dataset_factory
from tlt.models import model_factory
from tlt.utils.platform_util import CPUInfo, OptimizedPlatformUtil, PlatformUtil

# Specify the the default dataset directory
dataset_directory = os.environ["DATASET_DIR"] if "DATASET_DIR" in os.environ else \
    os.path.join(os.environ["HOME"], "dataset")

# Specify a directory for output (saved models and checkpoints)
output_directory = os.environ["OUTPUT_DIR"] if "OUTPUT_DIR" in os.environ else \
    os.path.join(os.environ["HOME"], "output")

# Location where Hugging Face will locally store data
os.environ['HF_HOME'] = dataset_directory

datasets.utils.logging.set_verbosity(datasets.logging.ERROR)

# Data Frame styles
table_styles =[{
    'selector': 'caption',
    'props': [
        ('text-align', 'center'),
        ('color', 'black'),
        ('font-size', '16px')
    ]
}]

# Colors used in charts
blue = '#0071c5'
dark_blue = '#003c71'
yellow = '#ffcc4d'

## 1. Display Platform Information

In [ ]:
# Get and display CPU/platform information
cpu_info = CPUInfo()
platform_util = PlatformUtil()
print("{0} CPU Information {0}".format("=" * 20))
print("CPU family:", platform_util.cpu_family)
print("CPU model:", platform_util.cpu_model)
print("CPU type:", platform_util.cpu_type)
print("Physical cores per socket:", cpu_info.cores_per_socket)
print("Total physical cores:", cpu_info.cores)
cpufreq = psutil.cpu_freq()
print("Max Frequency:", cpufreq.max)
print("Min Frequency:", cpufreq.min)
cpu_socket_count = cpu_info.sockets
print("Socket Number:", cpu_socket_count)

print("\n{0} Memory Information {0}".format("=" * 20))
svmem = psutil.virtual_memory()
print("Total: ", int(svmem.total / (1024 ** 3)), "GB")

# Display Hugging Face version information
print("\n{0} Hugging Face Information {0}".format("=" * 20))
print("Hugging Face Transformers version:", transformers.__version__)
print("Hugging Face Datasets version:", datasets.__version__)

# Display PyTorch version information
print("\n{0} PyTorch Information {0}".format("=" * 20))
print("PyTorch version:", torch.__version__)

## 2. Select a model and define parameters to use during training and evaluation

### Select a model

See the list of supported PyTorch text classification models from Hugging Face in the Intel Transfer Learning Tool.

In [ ]:
framework = 'pytorch'
use_case = 'text_classification'
model_hub = 'huggingface'
supported_models = model_factory.get_supported_models(framework, use_case)
supported_models = supported_models[use_case]

# Filter to only get relevant models
supported_models = { key:value for (key,value) in supported_models.items() if value[framework]['model_hub'] == model_hub}

print("Supported {} models for {} from {}".format(framework, use_case, model_hub))
print("=" * 70)
for model_name in supported_models.keys():
    print(model_name)

Set the `model_name` to the model that will be used during this experiment.

In [ ]:
# Select a model
model_name = "bert-base-cased"

### Select a dataset

For these experiments, we will be using text classification datasets from the [Hugging Face Datasets catalog](https://huggingface.co/datasets?task_categories=task_categories:text-classification&sort=downloads). Specify the name of the dataset to use with the `dataset_name` variable in the next cell.

In [ ]:
dataset_name = 'imdb'

### Define parameters

For consistency between the model training experiments using Hugging Face and the Intel Transfer Learning Tool, the next cell defines parameters that will be used by both methods.

In [ ]:
# Number of training epochs
training_epochs = 2

# Shuffle the files after each training epoch
shuffle_files = True

# Define the name of the split to use for validation (i.e. 'validation' or 'test')
eval_split=None
# If eval_split=None, split the 'train' dataset
validation_split = 0.05
training_split = 0.1

# Set seed for consistency between runs (or None)
seed = 10

# List of batch size(s) to compare (maximum of 4 batch sizes to try)
batch_size_list = [ 16, 32 ]

learning_rate=3e-5

# Text preprocessing
max_sequence_length = 128
padding = 'max_length'
truncation = True

# Use the Intel Extension for PyTorch
use_ipex = True

Validate parameter values and then print out the parameters.

In [ ]:
if not isinstance(training_epochs, int):
    raise TypeError("The training_epochs parameter should be an integer, but found a {}".format(type(training_epochs)))

if training_epochs < 1:
    raise ValueError("The training_epochs parameter should not be less than 1.")
    
if not isinstance(shuffle_files, bool):
    raise TypeError("The shuffle_files parameter should be a bool, but found a {}".format(type(shuffle_files)))

if not eval_split:
    if not isinstance(validation_split, float):
        raise TypeError("The validation_split parameter should be a float, but found a {}".format(type(validation_split)))

    if not isinstance(training_split, float):
        raise TypeError("The training_split parameter should be a float, but found a {}".format(type(training_split)))

if validation_split + training_split > 1:
    raise ValueError("The sum of validation_split and training_split should not be greater than 1.")

if seed and not isinstance(seed, int):
    raise TypeError("The seed parameter should be a integer or None, but found a {}".format(type(seed)))

if len(batch_size_list) > 4 or len(batch_size_list) == 0:
    raise ValueError("The batch_size_list should have at most 4 values, but found {} values ({})".format(
        len(batch_size_list), batch_size_list))
    
print("Number of training epochs:", training_epochs)
print("Shuffle files:", shuffle_files)
print("Training split: {}%".format('train' if eval_split else training_split*100))
print("Validation split: {}%".format(eval_split if eval_split else validation_split*100))
print("Seed:", str(seed))
print("Batch size list:", batch_size_list)

## 3. Train and evaluate the models

In this section, we will compare the time that it takes to fine tune the text classification model using the dataset that was selected in the previous section.

The fine tuning will be done in two different ways:
* Using the Hugging Face python libraries
* Using the Intel Transfer Learning Tool

### Fine tuning using the Hugging Face libraries

First, we download and prepare the dataset.

In [ ]:
transformers.set_seed(seed)

# Determine the splits to load
split = ['train']
if eval_split:
    split.append(eval_split)

# Load the dataset from the Hugging Face dataset catalog
hf_dataset = datasets.load_dataset(dataset_name, cache_dir=dataset_directory, split=split)

# Load the tokenizer based on our selected model
hf_tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, cache_dir=output_directory)

text_column_names = [col_name for col_name in hf_dataset[0].column_names if col_name != 'label' and
                     all(isinstance(s, str) for s in hf_dataset[0][col_name])]

# Tokenize the dataset
def tokenize_function(examples):
    args = (examples[text_column_name] for text_column_name in text_column_names)
    return hf_tokenizer(*args, padding=padding, max_length=max_sequence_length, truncation=truncation)

tokenized_hf_dataset = [d.map(tokenize_function, batched=True) for d in hf_dataset]
for tokenized in tokenized_hf_dataset:
    tokenized.set_format('torch')

# If eval_split is defined, that split will be used for validation. Otherwise, the 'train' dataset split will be
# split by the defined percentage to use for training and evaluation.
hf_train_dataset = tokenized_hf_dataset[0]
if eval_split:
    print("Using 'train' and '{}' dataset splits".format(eval_split))
    hf_train_subset = hf_train_dataset
    hf_eval_subset = tokenized_hf_dataset[1]
else:
    dataset_length = len(hf_train_dataset)
    train_size = int(training_split * dataset_length)
    eval_size = int(validation_split * dataset_length)
    generator = torch.Generator().manual_seed(seed) 
    dataset_indices = torch.randperm(dataset_length, generator=generator).tolist() if shuffle_files else range(dataset_length)
    train_indices = dataset_indices[:train_size]
    eval_indices = dataset_indices[train_size:train_size + eval_size]
    print("Using {}% for training and {}% for validation".format(training_split * 100, validation_split * 100))
    print("Total dataset size:", dataset_length)
    print("Train size:", train_size)
    print("Eval size:", eval_size)
    hf_train_subset = hf_train_dataset.select(train_indices)
    hf_eval_subset = hf_train_dataset.select(eval_indices)

# Get the number of classes from the train dataset features (either called 'label' or 'labels')
class_names = hf_dataset[0].features['label'].names if 'label' in hf_dataset[0].features else \
    hf_dataset[0].features['labels'].names
print("Class names: {}".format(class_names))

hf_train_dataset_length = len(hf_train_subset)
hf_eval_dataset_length = len(hf_eval_subset)

# Define function to compute accuracy to pass to the Hugging Face Trainer
def compute_metrics(p: transformers.EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

Next, we iterate through our list of batch sizes to train the model for each configuration with the dataset that was prepared in the previous cell. After training is complete, the model is evaluated and exported. The training and evaluation metrics are saved to lists.

In [ ]:
hf_saved_model_paths = []
hf_training_metrics = []
hf_eval_results = []

for i, batch_size in enumerate(batch_size_list):
    print('-' * 40)
    print('Training using batch size: {}'.format(batch_size))
    print('-' * 40)
    
    # Get the model from pretrained
    model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(class_names))

    # Setup a directory to save the model output
    saved_model_dir = os.path.join(output_directory, model_name, 'HF_model_bs{}'.format(batch_size))
    
    # Note: Even when setting do_eval=False, an error gets thrown if a eval_dataset is not provided
    training_args = transformers.TrainingArguments(
        output_dir=saved_model_dir,
        do_eval=False,
        do_train=True,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=training_epochs,
        evaluation_strategy="epoch",
        push_to_hub=False,
        no_cuda=True,
        overwrite_output_dir=True,
        seed=seed,
        data_seed=seed,
        use_ipex=use_ipex
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=hf_train_subset,
        eval_dataset=hf_eval_subset,
        compute_metrics=compute_metrics,
        tokenizer=hf_tokenizer,
    )

    # Train the model
    history = trainer.train()
    
    # Evaluate the model
    eval_results = trainer.evaluate()
    
    # Export the trained model
    trainer.save_model(saved_model_dir)
        
    # Save objects and metrics
    hf_training_metrics.append(history)
    hf_eval_results.append(eval_results)
    hf_saved_model_paths.append(saved_model_dir)

### Using the Intel Transfer Learning tool API

Next, we train the same model using the same parameters using the Intel Transfer Learning Tool. The Intel Transfer Learning Tool training method has an argument called `use_trainer` to determine if the Hugging Face Trainer will be used. If `use_trainer=False`, the torch libraries will be used to train the model. The next section will do the training with and without the Hugging Face Trainer, so that we can gather metrics both ways. After the model is trained, it is evaluted. Again, we save the training and evaluation metrics to lists.

In [ ]:
# Intel Transfer Learning Tool using the Hugging Face Trainer
tlt_trainer_saved_model_paths = []
tlt_trainer_training_metrics = []
tlt_trainer_eval_results = []

# Intel Transfer Learning Tool training using torch libraries
tlt_torch_saved_model_paths = []
tlt_torch_training_metrics = []
tlt_torch_eval_results = []

split_names = ['train'] if eval_split is None else ['train', eval_split]

for i, batch_size in enumerate(batch_size_list):
    print('-' * 40)
    print('Training using batch size: {}'.format(batch_size))
    print('-' * 40)

    # Get the dataset
    dataset = dataset_factory.get_dataset(dataset_directory, use_case, framework, dataset_name,
                                          dataset_catalog="huggingface", shuffle_files=shuffle_files,
                                          split=split_names)

    # Batch and tokenize the dataset
    dataset.preprocess(model_name, batch_size=batch_size, max_length=max_sequence_length, padding=padding,
                       truncation=truncation)

    # If the dataset doesn't have a defined split, then split it by percentages
    if eval_split is None:
        dataset.shuffle_split(train_pct=training_split, val_pct=validation_split)
        
    tlt_train_dataset_length = len(dataset.train_subset)
    
    if eval_split == 'test':
        eval_dataset = dataset.test_subset
        tlt_eval_dataset_length = len(eval_dataset)
    else:
        eval_dataset = dataset.validation_subset
        tlt_eval_dataset_length = len(eval_dataset)
    
    # Verify dataset length between the experiments
    assert tlt_train_dataset_length == hf_train_dataset_length
    assert tlt_eval_dataset_length == hf_eval_dataset_length
        
    for use_trainer in [True, False]:
        print('\nTraining using Hugging Face Trainer: {}'.format(use_trainer))
        
        # Get the model
        model = model_factory.get_model(model_name, framework)

        # Train the model
        history = model.train(dataset, output_directory, epochs=training_epochs, ipex_optimize=use_ipex,
                              use_trainer=use_trainer, do_eval=False, learning_rate=learning_rate, seed=seed)
        
        eval_metrics = model.evaluate(eval_dataset)

        # Save the model
        saved_model_dir = model.export(output_directory)
        
        if use_trainer:
            tlt_trainer_training_metrics.append(history)
            tlt_trainer_saved_model_paths.append(saved_model_dir)
            tlt_trainer_eval_results.append(eval_metrics)
        else:
            tlt_torch_training_metrics.append(history)
            tlt_torch_saved_model_paths.append(saved_model_dir)
            tlt_torch_eval_results.append(eval_metrics)

## 4. Compare metrics

This section compares metrics for training and evaluating the model for the following experiments:
* Using the Hugging Face Trainer
* Using the Intel Transfer Learning Tool with the Hugging Face Trainer
* Using the Intel Transfer Learning Tool with torch

In [ ]:
display_df = []

# Display training metrics
for i, batch_size in enumerate(batch_size_list):
    df = pd.DataFrame({
        'Hugging Face Trainer': [hf_training_metrics[i].metrics['train_loss'], hf_training_metrics[i].metrics['train_samples_per_second'], hf_training_metrics[i].metrics['train_runtime']],
        'Intel Transfer Learning Tool<br>using the HF Trainer': [tlt_trainer_training_metrics[i].metrics['train_loss'], tlt_trainer_training_metrics[i].metrics['train_samples_per_second'], tlt_trainer_training_metrics[i].metrics['train_runtime']],
        'Intel Transfer Learning Tool<br>using Torch': [tlt_torch_training_metrics[i]['Loss'], tlt_torch_training_metrics[i]['train_samples_per_second'][0], tlt_torch_training_metrics[i]['train_runtime'][0]],
    }, index = ['Loss', 'Samples per second', 'Train Runtime'])
    df = df.style.set_table_styles(table_styles).set_caption("Training metrics with batch size {}".format(batch_size))
    display_df.append(df)

# Display evaluation metrics
for i, batch_size in enumerate(batch_size_list):
    df = pd.DataFrame({
        'Eval using the<br>Hugging Face Trainer': ["{0:.2%}".format(hf_eval_results[i]['eval_accuracy']), hf_eval_results[i]['eval_loss'], hf_eval_results[i]['eval_samples_per_second'], hf_eval_results[i]['eval_runtime']],
        'Intel Transfer Learning Tool<br>eval using the HF Trainer': ["{0:.2%}".format(tlt_trainer_eval_results[i]['eval_accuracy']), tlt_trainer_eval_results[i]['eval_loss'], tlt_trainer_eval_results[i]['eval_samples_per_second'], tlt_trainer_eval_results[i]['eval_runtime']],
        'Intel Transfer Learning Tool<br>eval using Torch': ["{0:.2%}".format(tlt_torch_eval_results[i]['eval_accuracy']), tlt_torch_eval_results[i]['eval_loss'], tlt_torch_eval_results[i]['eval_samples_per_second'], tlt_torch_eval_results[i]['eval_runtime']],
    }, index = ['Eval accuracy', 'Eval Loss', 'Samples per second', 'Train Runtime'])
    df = df.style.set_table_styles(table_styles).set_caption("Training metrics with batch size {}".format(batch_size))
    display_df.append(df)
    
for df in display_df:
    display(df)

### Training metrics

Generate charts to compare the time that it took to train the model in each experiment (lower is better) and the thoughput (higher is better).

In [ ]:
# Bar chart group labels
groups = ["batch size = {}".format(bs) for bs in batch_size_list]

hf_train_runtime = [x.metrics['train_runtime'] for x in hf_training_metrics]
tlt_trainer_train_runtime = [x.metrics['train_runtime'] for x in tlt_trainer_training_metrics]
tlt_torch_train_runtime = [x['train_runtime'][0] for x in tlt_torch_training_metrics]

hf_train_throughput = [x.metrics['train_samples_per_second'] for x in hf_training_metrics]
tlt_trainer_train_throughput = [x.metrics['train_samples_per_second'] for x in tlt_trainer_training_metrics]
tlt_torch_train_thoughput = [x['train_samples_per_second'][0] for x in tlt_torch_training_metrics]

x = np.arange(len(groups))
width = 0.2  # the width of the bars
multiplier = 0

# Setup bars for training run times
fig, (ax1, ax2) = plt.subplots(2)
fig.set_figheight(15)
fig.set_figwidth(10)
rects_tf = ax1.bar(x, hf_train_runtime, width, label='HF trained', color=yellow)
rects_tlt_trainer = ax1.bar(x + width, tlt_trainer_train_runtime, width, label='TLT using Trainer', color=blue)
rects_tlt_torch = ax1.bar(x + width * 2, tlt_torch_train_runtime, width, label='TLT using Torch', color=dark_blue)
ax1.bar_label(rects_tf, padding=3)
ax1.bar_label(rects_tlt_trainer, padding=3)
ax1.bar_label(rects_tlt_torch, padding=3)

# Add labels, title, and legend
ax1.set_ylabel('Seconds')
ax1.set_title('Training Runtime')
ax1.set_xticks(x+width, groups)
ax1.set_ymargin(0.2) 
ax1.legend(ncols=2)

# Setup bars for throughput
rects_tf = ax2.bar(x, hf_train_throughput, width, label='HF trained', color=yellow)
rects_tlt_trainer = ax2.bar(x + width, tlt_trainer_train_throughput, width, label='TLT using Trainer', color=blue)
rects_tlt_torch = ax2.bar(x + width * 2, tlt_torch_train_thoughput, width, label='TLT using Torch', color=dark_blue)
ax2.bar_label(rects_tf, padding=3)
ax2.bar_label(rects_tlt_trainer, padding=3)
ax2.bar_label(rects_tlt_torch, padding=3)

# Add labels, title, and legend
ax2.set_ylabel('Samples per second')
ax2.set_title('Training Throughput')
ax2.set_xticks(x+width, groups)
ax2.set_ymargin(0.2) 
ax2.legend(ncols=2)
plt.show()

### Evaluation Metrics

Next, we generate charts to compare the evaluation metrics for the same three experiments that were used for training the model. We have a charts with the validation accuracy, total evaluation time (lower is better), and evaluation throughput (higher is better).

In [ ]:
# Decimals used for rounding
decimals = 2

# Get the evaluation metrics
hf_eval_acc = [round(x['eval_accuracy'] * 100, decimals) for x in hf_eval_results]
tlt_trainer_eval_acc = [round(x['eval_accuracy'] * 100, decimals) for x in tlt_trainer_eval_results]
tlt_torch_eval_acc = [round(x['eval_accuracy'] * 100, decimals) for x in tlt_torch_eval_results]

hf_eval_runtime = [round(x['eval_runtime'], decimals) for x in hf_eval_results]
tlt_trainer_eval_runtime = [round(x['eval_runtime'], decimals) for x in tlt_trainer_eval_results]
tlt_torch_eval_runtime = [round(x['eval_runtime'], decimals) for x in tlt_torch_eval_results]

hf_eval_throughput = [round(x['eval_samples_per_second'], decimals) for x in hf_eval_results]
tlt_trainer_eval_throughput = [round(x['eval_samples_per_second'], decimals) for x in tlt_trainer_eval_results]
tlt_torch_eval_thoughput = [round(x['eval_samples_per_second'], decimals) for x in tlt_torch_eval_results]

x = np.arange(len(groups))
width = 0.2  # the width of the bars
multiplier = 0

# Setup bars for training run times
fig, (ax1, ax2, ax3) = plt.subplots(3)
fig.set_figheight(20)
fig.set_figwidth(10)
rects_tf = ax1.bar(x, hf_eval_acc, width, label='HF evaluated', color=yellow)
rects_tlt_trainer = ax1.bar(x + width, tlt_trainer_eval_acc, width, label='TLT eval using Trainer', color=blue)
rects_tlt_torch = ax1.bar(x + width * 2, tlt_torch_eval_acc, width, label='TLT eval using Torch', color=dark_blue)
ax1.bar_label(rects_tf, padding=3)
ax1.bar_label(rects_tlt_trainer, padding=3)
ax1.bar_label(rects_tlt_torch, padding=3)

# Add labels, title, and legend
ax1.set_ylabel('Percentage (%)')
ax1.set_title('Evaluation Accuracy')
ax1.set_xticks(x+width, groups)
ax1.set_ymargin(0.2) 
ax1.legend(ncols=2)

rects_tf = ax2.bar(x, hf_eval_runtime, width, label='HF evaluated', color=yellow)
rects_tlt_trainer = ax2.bar(x + width, tlt_trainer_eval_runtime, width, label='TLT eval using Trainer', color=blue)
rects_tlt_torch = ax2.bar(x + width * 2, tlt_torch_eval_runtime, width, label='TLT eval using Torch', color=dark_blue)
ax2.bar_label(rects_tf, padding=3)
ax2.bar_label(rects_tlt_trainer, padding=3)
ax2.bar_label(rects_tlt_torch, padding=3)

# Add labels, title, and legend
ax2.set_ylabel('Seconds')
ax2.set_title('Evaluation Runtime')
ax2.set_xticks(x+width, groups)
ax2.set_ymargin(0.2) 
ax2.legend(ncols=2)

# Setup bars for throughput
rects_tf = ax3.bar(x, hf_eval_throughput, width, label='HF evaluated', color=yellow)
rects_tlt_trainer = ax3.bar(x + width, tlt_trainer_eval_throughput, width, label='TLT eval using Trainer', color=blue)
rects_tlt_torch = ax3.bar(x + width * 2, tlt_torch_eval_thoughput, width, label='TLT eval using Torch', color=dark_blue)
ax3.bar_label(rects_tf, padding=3)
ax3.bar_label(rects_tlt_trainer, padding=3)
ax3.bar_label(rects_tlt_torch, padding=3)

# Add labels, title, and legend
ax3.set_ylabel('Samples per second')
ax3.set_title('Evaluation Throughput')
ax3.set_xticks(x+width, groups)
ax3.set_ymargin(0.2) 
ax3.legend(ncols=2)
plt.show()

## Next Steps

This concludes our performance comparison using a Hugging Face model with the Hugging Face `Trainer` and the Intel Transfer Learning Tool. All of the models trained during these experiments have been saved to your output directory. Any of these can be loaded back to perform further experiments.

In [ ]:
for i, batch_size in enumerate(batch_size_list):
    print("\nUsing batch size {}".format(batch_size))
    print('-' * 25)
    print("Model trained using the Hugging Face Trainer:\n\t", hf_saved_model_paths[i])
    print("Model trained using the Intel Transfer Learning tool and the Hugging Face Trainer:\n\t", tlt_trainer_saved_model_paths[i])
    print("Model trained using the Intel Transfer Learning tool and the PyTorch libraries:\n\t", tlt_torch_saved_model_paths[i])

## Citations 

```
@misc{devlin2019bert,
      title={BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding}, 
      author={Jacob Devlin and Ming-Wei Chang and Kenton Lee and Kristina Toutanova},
      year={2019},
      eprint={1810.04805},
      archivePrefix={arXiv},
      primaryClass={cs.CL}
}
```

```
@InProceedings{maas-EtAl:2011:ACL-HLT2011,
  author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
  title     = {Learning Word Vectors for Sentiment Analysis},
  booktitle = {Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies},
  month     = {June},
  year      = {2011},
  address   = {Portland, Oregon, USA},
  publisher = {Association for Computational Linguistics},
  pages     = {142--150},
  url       = {http://www.aclweb.org/anthology/P11-1015}
}
```